In [583]:
import algosdk
import math
import pandas as pd

In [584]:
def connect_testnet():
    algod_address_testnet = "https://testnet-algorand.api.purestake.io/ps1"
    port_test = ""
    token_test = {
       "X-API-Key": "B3SU4KcVKi94Jap2VXkK83xx38bsv95K5UZm2lab",
    }

    algod_client_test = algosdk.algod.AlgodClient(port_test, algod_address_testnet, token_test)
    return algod_client_test

In [585]:
def connect_mainnet():
    algod_address_mainnet = "https://mainnet-algorand.api.purestake.io/ps1"
    port = ""
    purestake_api_key = '94IaBae3fC56OeZtiWIY03TL5K4qeQjE5thyZbF0'
    token = {
        'X-API-key' : purestake_api_key,
    }

    algod_client = algosdk.algod.AlgodClient(port, algod_address_mainnet, token) # Initialize the algod client and get the basic parameters
    return algod_client

# if using my own node...
#token = "token from the file algod.token"
#server = "http://127.0.0.1" #localhost
#port = 8081 # default

In [586]:
def client_check(algod_client):
    try:
        status = algod_client.status()
    except Exception as e:
        print("Failed to get algod status: {}".format(e))

    if status:
        print("algod last round: {}".format(status.get("lastRound")))
        print("algod time since last round: {}".format(status.get("timeSinceLastRound")))
        print("algod catchup: {}".format(status.get("catchupTime")))
        print("algod latest version: {}".format(status.get("lastConsensusVersion")))

    # Retrieve latest block information                                                                                                                                               
    last_round = algod_client.status().get("lastRound")
    print("####################")
    block = algod_client.block_info(last_round)
    print(block)

In [587]:
#algod_client_test = connect_testnet()
#client_check(algod_client_test)

In [588]:
algod_client = connect_mainnet()
client_check(algod_client)

algod last round: 5769295
algod time since last round: 3010442986
algod catchup: 0
algod latest version: https://github.com/algorandfoundation/specs/tree/4a9db6a25595c6fd097cf9cc137cc83027787eaa
####################
{'hash': 'EQ2JRHP6JSWYNBXDWO4I4KJIFKRJCD2PEDKQOHIA3CBSHWXDU3IQ', 'previousBlockHash': 'MBYDEYKUJLLC7ON52SS7JBYJJFO4UKUZ7GYLRWISBXDTU5CE2ARA', 'seed': 'EHFHNGUGEGGQF3OKWWJSSXJKZV2JGYFBGH5CDG6UU7D42XA6X6OQ', 'proposer': 'I3345FUQQ2GRBHFZQPLYQQX5HJMMRZMABCHRLWV6RCJYC6OO4MOLEUBEGU', 'round': 5769295, 'period': 0, 'txnRoot': 'WRS2VL2OQ5LPWBYLNBCZV3MEQ4DACSRDES6IUKHGOWYQERJRWC5A', 'reward': 113975, 'rate': 25999980, 'frac': 2256079096, 'txns': {}, 'timestamp': 1585599255, 'currentProtocol': 'https://github.com/algorandfoundation/specs/tree/4a9db6a25595c6fd097cf9cc137cc83027787eaa', 'nextProtocol': '', 'nextProtocolApprovals': 0, 'nextProtocolVoteBefore': 0, 'nextProtocolSwitchOn': 0, 'upgradePropose': '', 'upgradeApprove': False}


In [596]:
# For retrieving the real covid data from mainnet

address = "COVIDR5MYE757XMDCFOCS5BXFF4SKD5RTOF4RTA67F47YTJSBR5U7TKBNU"
fromRound = 5646000
maxTxnPerCall = 500 # max transactions in a batch
batchSize = 512 #Read the transactions from the blockchain in 512-block installations

params = algod_client.suggested_params()
print("total rounds:",params['lastRound'] - fromRound)
lastRound_testing = fromRound + 20*batchSize
print("testing rounds:",lastRound_testing - fromRound)

total rounds: 123361
testing rounds: 10240


In [590]:
#  A recursive function for getting a batch of transactions, to overcome
# the limitation of maxTxnPerCall transaction per call to the API

def getTransactionBatch(fromRnd, toRnd):
    
    if (fromRnd > toRnd):# sanity check
        return []
    
    txs = algod_client.transactions_by_address(address,fromRnd,toRnd,maxTxnPerCall) # make an API call to get the transactions
    
    return txs['transactions']
    
    '''
    # If we got all the transactions, just return them
    if (fromRnd == toRnd | len(txs['transactions']) < maxTxnPerCall):
        return txs['transactions']
    
        # FIXME: If a single block contains more than maxTxnPerCall
        # transactions for the target address, the code above will return
        # only maxTxnPerCall of them.
        # This is an unlikely case, and not easy to handle. The only way to
        # handle it is to call algod.block(round#), then go over all the
        # transactions in this block and take only the ones corresponding
        # to the target address.
    
    else: # recursive call to get them in two smaller chunks
        midRnd = math.floor((fromRnd+toRnd) / 2)
        txns1 = getTransactionBatch(fromRnd, midRnd)
        txns2 = getTransactionBatch(midRnd+1, toRnd)
        # return the concatenation of the two chunks
        return txns1.concat(txns2)
    '''

In [598]:
lastRound = params['lastRound']
#lastRound = lastRound_testing
txns = getTransactionBatch(fromRound, lastRound) # Fetch transactions for these rounds 
print("found {} transactions".format(len(txns)))

found 500 transactions


In [600]:
def decode_node_data(transactions):
    
    cols = ['round','country_code','region_code','zip_code_3dig',
            'gender','was_quarantined','reason_quarantine','status_quarantine',
            'date_start_quarantine','date_end_quarantine',
            'is_symptomatic','fever','cough','difficulty_breathing','fatigue','sore_throat',
            'date_start_symptoms','date_end_symptoms','still_symptomatic',
            'got_medical_care',
            'type_of_care',
            'hospitalized','date_when_admitted','date_when_discharged','still_in_hospital',
            'tested','tried_to_get_tested','was_denied','date_test','results_test','location_test']
        
    trans_df = pd.DataFrame(columns = cols)
    
    for i in range(len(transactions)):
        
        tx_dict = transactions[i]
        encoded_note = tx_dict['noteb64']
        noteData = str(algosdk.encoding.base64.b64decode(encoded_note)).encode('utf8').decode('utf8')
        
        #print(noteData)
        noteData_list = noteData[7:].split('\\')
        #print("for round {}".format(tx_dict['round']))
        
        df_data_list = ['NaN']*len(cols)
        df_data_list[0] = tx_dict['round']
        
        decode_chars = ['xa1','xa2','xa3','xa6','xa7','xaa']
        clean_noteData = []
        
        for item in noteData_list:
            for char in decode_chars:
                if char in item:
                    item = item.replace(char,'')
            clean_noteData.append(item)
                    
                
        #print(noteData_list) 
        #print('\n')
        #print(clean_noteData)
        
        for j in range(0,len(clean_noteData)-1):
            if 'gc' in clean_noteData[j]:
                df_data_list[1] = clean_noteData[j+1]
            elif 'gr' in clean_noteData[j]:
                df_data_list[2] = clean_noteData[j+1]
            elif 'gzp' in clean_noteData[j]:
                df_data_list[3] = clean_noteData[j+1]
            elif 'gs' in clean_noteData[j]:
                df_data_list[4] = clean_noteData[j+1]
                
                
            elif 'qz' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[5] = True
                if clean_noteData[j+1].replace('x','') == '-01':
                    df_data_list[5] = False
            
            elif 'q1' in clean_noteData[j]:
                df_data_list[6] = 'due to symptoms'
            elif 'q2' in clean_noteData[j]:
                df_data_list[6] = 'voluntary'
            elif 'q3' in clean_noteData[j]:
                df_data_list[6] = 'personally required'
            elif 'q4' in clean_noteData[j]:
                df_data_list[6] = 'general quarantine'
                
            elif 'qdn' in clean_noteData[j]:
                df_data_list[7] = 'still quarantined'
                
            elif 'qds' in clean_noteData[j]:
                df_data_list[8] = clean_noteData[j+1]
            
            elif 'qde' in clean_noteData[j]:
                df_data_list[9] = clean_noteData[j+1]
                
            elif 'sz' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[10] = True
                if clean_noteData[j+1].replace('x','') == '-01':
                    df_data_list[10] = False
                
            elif 's1' in clean_noteData[j]:
                df_data_list[11] = True
            elif 's2' in clean_noteData[j]:
                df_data_list[12] = True
            elif 's3' in clean_noteData[j]:
                df_data_list[13] = True
            elif 's4' in clean_noteData[j]:
                df_data_list[14] = True
            elif 's5' in clean_noteData[j]:
                df_data_list[15] = True
                
            elif 'sds' in clean_noteData[j]:
                df_data_list[16] = clean_noteData[j+1]
                
            elif 'sde' in clean_noteData[j]:
                df_data_list[17] = clean_noteData[j+1]
                
            elif 'sdn' in clean_noteData[j]:
                df_data_list[18] = True
                
            elif 'mz' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[19] = True
                if clean_noteData[j+1].replace('x','') == '-01':
                    df_data_list[19] = False
                    
            elif 'm1' in clean_noteData[j]:
                df_data_list[20] = 'doctors office'
            elif 'm2' in clean_noteData[j]:
                df_data_list[20] = 'walk-in clinic'
            elif 'm3' in clean_noteData[j]:
                df_data_list[20] = 'virtual care'
            elif 'm4' in clean_noteData[j]:
                df_data_list[20] = 'hospital/ER'
            elif 'm5' in clean_noteData[j]:
                df_data_list[20] = 'other'
                
            elif 'mh' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[21] = True
                if clean_noteData[j+1].replace('x','') == '-01':
                    df_data_list[21] = False
                    
            elif 'mhs' in clean_noteData[j]:
                df_data_list[22] = clean_noteData[j+1]
            
            elif 'mhe' in clean_noteData[j]:
                df_data_list[23] = clean_noteData[j+1]
            
            elif 'mhn' in clean_noteData[j]:
                df_data_list[24] = True
            
            elif 'tz' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[25] = True 
                if clean_noteData[j+1].replace('x','') == '-01':
                    df_data_list[25] = False
                    
            elif 'tt' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[26] = True
                if clean_noteData[j+1].replace('x','') == '-01':
                    df_data_list[26] = False
                if clean_noteData[j+1].replace('x','') == '02':
                    df_data_list[27] = True
            
            elif 'td' in clean_noteData[j]:
                df_data_list[28] = clean_noteData[j+1]
                
            elif 'tr' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '-01':
                    df_data_list[29] = 'negative'
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[29] = 'positive'
                if clean_noteData[j+1].replace('x','') == '02':
                    df_data_list[29] = 'waiting'                
                    
            elif 'tl' in clean_noteData[j]:
                if clean_noteData[j+1].replace('x','') == '01':
                    df_data_list[30] = 'Dr. Office'                
                if clean_noteData[j+1].replace('x','') == '02':
                    df_data_list[30] = 'Hospital'
                if clean_noteData[j+1].replace('x','') == '03':
                    df_data_list[30] = 'Urgent Care'                    
                if clean_noteData[j+1].replace('x','') == '04':
                    df_data_list[30] = 'Ad-hoc center'                    
                if clean_noteData[j+1].replace('x','') == '05':
                    df_data_list[30] = 'Other'
                    
            
        #print('\n')
        trans_df.loc[i] = df_data_list
    return trans_df

In [602]:
pd.set_option('display.max_rows', None)
data_df = decode_node_data(txns)
display(data_df)

,round,country_code,region_code,zip_code_3dig,gender,was_quarantined,reason_quarantine,status_quarantine,date_start_quarantine,date_end_quarantine,...,hospitalized,date_when_admitted,date_when_discharged,still_in_hospital,tested,tried_to_get_tested,was_denied,date_test,results_test,location_test
0,5768924,RU,NaN,NaN,m,True,general quarantine,still quarantined,2020-03-30,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5768765,US,NaN,021,f,True,voluntary,still quarantined,2020-03-13,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5768436,US,TX,762,f,True,voluntary,still quarantined,2020-03-19,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5768305,US,CA,904,m,True,general quarantine,still quarantined,2020-03-18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5767876,US,MA,NaN,m,True,voluntary,still quarantined,2020-03-09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5767565,FR,IDF,NaN,f,True,general quarantine,still quarantined,2020-03-17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5766568,BE,WAL,NaN,m,True,general quarantine,still quarantined,2020-03-14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5766481,CI,01,NaN,m,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5766480,SE,AB,NaN,f,True,voluntary,still quarantined,2020-03-14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5766403,SE,AB,NaN,m,True,voluntary,still quarantined,2020-03-17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [603]:
data_df.to_csv('covidData.csv',index=False)

In [683]:
data_df_reduced = data_df[data_df['was_quarantined']!= 'NaN']
data_df_reduced = data_df_reduced[data_df_reduced['country_code'] == 'US']

In [684]:
# use anon 3 digit zipcode to try and impute state abbrevs
def long_lat(df):
    lat_long_df = pd.read_csv(r'C:\Users\rayzc\Downloads\us-zip-code-latitude-and-longitude.csv',sep=';')
    states = []
    for state,area in zip(df.region_code,df.zip_code_3dig):
        region = state
        if (region=='NaN') & (area!='NaN'):
            for s,z in zip(lat_long_df.State,lat_long_df.Zip):
                if area == str(z)[0:3]:
                    region = s
                    print(area,'matched with',region)
                    break
        states.append(region)

    df['state_code'] = states
    
    return df

In [685]:
data_df_reduced = long_lat(data_df_reduced)
data_df_reduced = data_df_reduced[data_df_reduced['state_code'] != 'NaN']

805 matched with NJ
317 matched with GA
112 matched with NY
101 matched with NY
805 matched with NJ
805 matched with NJ
805 matched with NJ
908 matched with CA
300 matched with GA
329 matched with FL
945 matched with CA
300 matched with GA
305 matched with GA
530 matched with WI
334 matched with FL
373 matched with TN
131 matched with NY
300 matched with GA
305 matched with GA
100 matched with NY
100 matched with NY
300 matched with GA
755 matched with TX
212 matched with MD
212 matched with MD
210 matched with MD
329 matched with FL
452 matched with OH
210 matched with MD
801 matched with NJ
286 matched with NC
100 matched with NY
105 matched with NY
212 matched with MD
921 matched with CA
212 matched with MD
100 matched with NY
100 matched with NY
200 matched with DC
117 matched with NY
112 matched with NY
333 matched with FL
112 matched with NY
112 matched with NY
112 matched with NY
105 matched with NY


In [691]:
data_df_reduced[['country_code','state_code','zip_code_3dig','gender','was_quarantined','reason_quarantine','date_start_quarantine','date_end_quarantine',
                'is_symptomatic','fever','cough','difficulty_breathing','fatigue','sore_throat','date_start_symptoms','date_end_symptoms']].reset_index(drop=True)

,country_code,state_code,zip_code_3dig,gender,was_quarantined,reason_quarantine,date_start_quarantine,date_end_quarantine,is_symptomatic,fever,cough,difficulty_breathing,fatigue,sore_throat,date_start_symptoms,date_end_symptoms
0,US,TX,762,f,True,voluntary,2020-03-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US,CA,904,m,True,general quarantine,2020-03-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,US,MA,NaN,m,True,voluntary,2020-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US,CA,947,f,True,general quarantine,2020-03-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,US,NC,272,f,True,voluntary,2020-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,US,RI,028,f,True,voluntary,2020-03-11,2020-03-25,True,True,NaN,NaN,True,NaN,2020-03-11,2020-03-25
6,US,MD,212,m,True,general quarantine,2020-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,US,CO,805,f,True,general quarantine,2020-03-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,US,NY,112,m,True,voluntary,2020-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,US,NJ,805,f,True,general quarantine,2020-03-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [690]:
data_df_reduced[['country_code','state_code','zip_code_3dig','gender','was_quarantined','reason_quarantine',
                 'date_start_quarantine','date_end_quarantine',
                'is_symptomatic','fever','cough','difficulty_breathing','fatigue',
                 'sore_throat','date_start_symptoms',
                 'date_end_symptoms']].reset_index(drop=True).to_csv('covidData_USonly_wState.csv',index=False)

In [724]:
reason_quarantine_bystate = pd.DataFrame(data_df_reduced[['state_code','reason_quarantine']].groupby('state_code')['reason_quarantine'].value_counts().unstack()).fillna(0)
gender_count_bystate = pd.DataFrame(data_df_reduced[['state_code','gender']].groupby('state_code')['gender'].value_counts().unstack())[['f','m']].fillna(0)
basic_visualization_df = reason_quarantine_bystate.merge(gender_count_bystate,on ='state_code')
basic_visualization_df.to_csv('covidData_USonly_wState_mappingbasic.csv',index=False)

In [ ]:
"""
rowcnt = 0

#fstr = fs.open_fs('covidData.csv')
#fstr.write('sr,rnd,'+headers.toString()+'\n') # write the header

# 'sr' is serial number, 'rnd' is round number

for rnd in range(fromRound, lastRound):
    toRnd = rnd + batchSize
    if (toRnd > lastRound):
        toRnd = lastRound
        
    txns = getTransactionBatch(rnd, toRnd) # Fetch transactions for these rounds 
    
    print(txns)

    # Write transaction
    for i in range(0,len(txns)):
        # Decode data in Note field
        tx = txns[i]
        noteData = algosdk.decodeObj(tx.note)

        #must have a serial number and data, else continue to next one
        if (noteData.s =='' | noteData.d == ''):
            continue
        line = noteData.s+','+tx.round
        for j in range(0,len(headers)):
            line += ","                 # followed by a comma
            key = headers[j]
            if (noteData.d[key]):       # if a value exists
                line += noteData.d[key] # write it
                
        #fstr.write(line+'\n');
        rowcnt+=1    
    rnd+=batchSize
"""

In [268]:
# attribute array
headers = [
    '_t',  
    '_v',  

    # general demographic data
    'gc',  # string, country code (see Location Data section below)
    'gr',  # string, region code  (see Location Data section below)
    'gzp', # string, 3-digit zip code (US only)
    'ga',  # integer, age group, if present must be in 1,11,21,31,41,51,56,61,66,71,76,81,85
    'gs',  # string , gender, if present must be 'm','f'

    # symptoms
    'sz',  # integer, is symptomatic, no-answer=0/no=-1/yes=1
    's1',  # boolean, fever
    's2',  # boolean, cough
    's3',  # boolean, difficulty breathing
    's4',  # boolean, fatigue
    's5',  # boolean, sore throat
    'sds', # date, when symptoms started, yyyy-mm-dd
    'sde', # date, when symptoms ended, yyyy-mm-dd
    'sdn', # boolean, still symptomatic

    #tested
    'tz',  # integer, tested, no-answer=0/no=-1/yes=1
    'tt',  # integer, tried to get tested, no=-1, yes=1, yes but was denied=2
    'td',  # date, test date, yyyy-mm-dd
    'tr',  # integer, test results, -1=negative,1=positive,2=waiting for result
    'tl',  #  integer, test location, 1=Dr office/2=Hospital/3=Urgent care/4=Ad-hoc center/5=Other

    # medical care
    'mz',  #  integer, received care, no-answer=0/no=-1/yes=1
    'm1',  #  boolean, doctor's office
    'm2',  #  boolean, walk-in clinic
    'm3',  #  boolean, virtual care
    'm4',  #  boolean, hospital/ER
    'm5',  #  boolean, other
    'mh',  #  integer, hospitalized, no-answer=0/no=-1/yes=1
    'mhs', #  date, when admitted, yyyy-mm-dd
    'mhe', #  date, when discharged, yyyy-mm-dd
    'mhn', #  boolean, still in hospital

    # quarantine
    'qz',  # integer, was quarantined, no-answer=0/no=-1/yes=1
    'q1',  # boolean, due to symptoms
    'q2',  # boolean, voluntarily
    'q3',  # boolean, personally required
    'q4',  # boolean, general quarantine
    'qds', # date, when quarantine started, yyyy-mm-dd
    'qde', # date, when quarantine ended, yyyy-mm-dd
    'qdn', # boolean, still quarantined
    'ql',  # integer, left quarantine temporarily no-answer=0/no=-1/yes=1

    'consent' # boolean' , user's consent, mandatory, must be 'true'
]